In [1]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl
from keras.models import Model
from keras import backend as K
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
sess = tf.Session()

/home/exacon03/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training = np.genfromtxt('/home/exacon03/Downloads/Sentiment Analysis Dataset.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None,max_rows=100000)

train_x = [x[1] for x in training]
train_y = np.asarray([x[0] for x in training])

/home/exacon03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [3]:
def save_model_for_production(model, version, path='prod_models'):
    tf.keras.backend.set_learning_phase(1)
    if not os.path.exists(path):
        os.mkdir(path)
    export_path = os.path.join(
        tf.compat.as_bytes(path),
        tf.compat.as_bytes(version))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    model_input = tf.saved_model.utils.build_tensor_info(model.input)
    model_output = tf.saved_model.utils.build_tensor_info(model.output)

    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'inputs': model_input},
            outputs={'output': model_output},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    with tf.keras.backend.get_session() as sess:
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:prediction_signature,
            })

        builder.save()

In [4]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=1, activation='sigmoid'))
    return model

In [5]:
# Create keyword arguments to pass to the 'tf-idf' vectorizer.
kwargs = {
            'ngram_range': (1,2),  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': 'word',  # Split text into word tokens.
            'min_df':  2,
}
vectorizer = TfidfVectorizer(**kwargs)

# Learn vocabulary from training texts and vectorize training texts.
train_x = vectorizer.fit_transform(train_x)

# print(type(train_x))

# Select top 'k' of the vectorized features.
selector = SelectKBest(chi2, k=min(20000, train_x.shape[1]))
train_x = selector.fit_transform(train_x, train_y).astype('float32')

In [6]:
# Defining the parameters for the model
layers=2
units=32
learning_rate=1e-3
dropout_rate=0.2
input_shape=train_x.shape[1:]
num_classes=2


# Create model instance.
model=mlp_model(layers,units,dropout_rate,input_shape,num_classes)

model.compile(loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
  metrics=['accuracy'])

In [8]:
# Train and validate model.
model.fit(train_x, train_y,
  batch_size=512,
  epochs=20,
  verbose=1,
  validation_split=0.2,
  callbacks = [EarlyStopping(monitor='val_loss', patience=1)],     
  shuffle=True)

Train on 80000 samples, validate on 20000 samples
Epoch 1/20
80000/80000 [==============================] - 14s 176us/step - loss: 0.3658 - acc: 0.8233 - val_loss: 0.4226 - val_acc: 0.8087
Epoch 2/20
80000/80000 [==============================] - 14s 175us/step - loss: 0.3634 - acc: 0.8250 - val_loss: 0.4218 - val_acc: 0.8094
Epoch 3/20
80000/80000 [==============================] - 14s 178us/step - loss: 0.3587 - acc: 0.8252 - val_loss: 0.4222 - val_acc: 0.8103


In [9]:
model.save('/home/exacon03/Jupyter/Kerasmodel.h5')

In [ ]:
while 1:
    evalSentence =input('Input a sentence to be evaluated, or Enter to quit: ')

    if len(evalSentence) == 0:
        break

    # Format your input for the neural net
    evalSentence=[evalSentence]
    tx = vectorizer.transform(evalSentence).astype('float32')
    tx = selector.transform(tx).astype('float32')
    pred = model.predict(tx)
    if pred[0][0]>0.5:
        print("Sentiment: Positive  Confidence Level:",int(np.round((pred[0][0]-0.5)*200)),"%")
    else:
        print("Sentiment: Negative  Confidence Level:",int(np.round((0.5-pred[0][0])*200)),"%")

Input a sentence to be evaluated, or Enter to quit: bad
Sentiment: Negative  Confidence Level: 47 %


In [ ]:
save_model_for_production(model, "1", "/home/exacon03/Jupyter/")